In [1]:
pip install --upgrade "watson-developer-cloud>=2.5.1"

    100% |████████████████████████████████| 286kB 7.4MB/s 
    100% |████████████████████████████████| 204kB 30.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/f3/1d/4f/5a5432c03a3c28c79e41bbf7110b871a88bab10393906b86fe
Successfully built watson-developer-cloud


In [0]:
import json
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, SentimentOptions, EntitiesOptions, KeywordsOptions

In [0]:
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2018-11-16',
    iam_apikey='-7fqyxZT99HP6OixEJbc4vF6GiF9Uzb78ltTl3OEf4Ou',
    url='https://gateway.watsonplatform.net/natural-language-understanding/api'
)

In [0]:
response = natural_language_understanding.analyze(
    url='www.wsj.com/news/markets',
    features=Features(sentiment=SentimentOptions(targets=['stocks']))).get_result()


In [5]:
print(json.dumps(response, indent=2))

{
  "usage": {
    "text_units": 1,
    "text_characters": 1536,
    "features": 1
  },
  "sentiment": {
    "targets": [
      {
        "text": "stocks",
        "score": -0.925061,
        "label": "negative"
      }
    ],
    "document": {
      "score": -0.507698,
      "label": "negative"
    }
  },
  "retrieved_url": "https://www.wsj.com/news/markets",
  "language": "en"
}


In [0]:
import pandas as pd

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/mod.csv")
data = data.drop(data.columns[0], axis=1)
data = data.dropna()
X = data['text'].values
y = data['sentiment'].values
data = data.drop(['sentiment'], axis = 1)
data.head(5)

,text
0,"Thank you, Ellen. We have a strong 2018, with ..."
1,Stock-based compensation totaled $2.3 billion....
2,"For opening remarks, I will be turning the cal..."
3,[Operator Instructions]. Our first question wi...
4,"Hey, thanks. So I guess the question I have an..."


In [9]:
score = 0 
a = 0

with open('watson_json.txt', 'w') as outfile:
    for i in data['text']:
        response = natural_language_understanding.analyze(
        text= i ,
        features=Features(
            entities=EntitiesOptions(emotion=False, sentiment=True, limit=2),
            keywords=KeywordsOptions(emotion=False, sentiment=True, limit=2)),language = 'en'
        ).get_result()

        a += 1 

        print(json.dumps(response, indent=2))
        json.dump(response, outfile)
        if response['entities']:
            score = score + response['entities'][0]['sentiment']['score']
        elif response['keywords']:
            score = score + response['keywords'][0]['sentiment']['score']
        else:
            score += 0

{
  "usage": {
    "text_units": 1,
    "text_characters": 271,
    "features": 2
  },
  "language": "en",
  "keywords": [
    {
      "text": "total revenues",
      "sentiment": {
        "score": 0.997936,
        "label": "positive"
      },
      "relevance": 0.948795,
      "count": 1
    },
    {
      "text": "benefit of our ongoing investments",
      "sentiment": {
        "score": 0.997936,
        "label": "positive"
      },
      "relevance": 0.874435,
      "count": 1
    }
  ],
  "entities": [
    {
      "type": "Person",
      "text": "Ellen",
      "sentiment": {
        "score": 0.553561,
        "label": "positive"
      },
      "relevance": 0.926911,
      "count": 1
    },
    {
      "type": "Quantity",
      "text": "$136.8 billion",
      "sentiment": {
        "score": 0.0,
        "label": "neutral"
      },
      "relevance": 0.926911,
      "count": 1
    }
  ]
}
{
  "usage": {
    "text_units": 1,
    "text_characters": 332,
    "features": 2
  },
  "lan

In [10]:
!ls

drive  model_bow.h5  sample_data  vocab.txt  watson_json.txt


In [0]:
score = score / a

In [12]:
score

0.20662283262325012